Bayesian GPLVM with full rank covariance
--
This notebook shows the difference between a full rank and a diagonal variational distribution over the latent values.

In [1]:
import GPflow
from GPflow import ekernels
from GPflow import kernels
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('ggplot')
%matplotlib inline
import pods
pods.datasets.overide_manual_authorize = True  # dont ask to authorize
np.random.seed(42)
GPflow.settings.numerics.quadrature = 'error'  # throw error if quadrature is used for kernel expectations

### Data
Install the oil dataset.

In [2]:
data = pods.datasets.oil_100()
Y = data['X']
print('Number of points X Number of dimensions', Y.shape)
data['citation']

Number of points X Number of dimensions (100, 12)


'Bishop, C. M. and G. D. James (1993). Analysis of multiphase flows using dual-energy gamma densitometry and neural networks. Nuclear Instruments and Methods in Physics Research A327, 580-593'

### Model construction
Create Bayesian GPLVM model using additive kernel.

In [3]:
Q = 3
M = 20  # number of inducing pts
N = Y.shape[0]
X_mean = GPflow.gplvm.PCA_reduce(Y, Q) # Initialise via PCA
Z = np.random.permutation(X_mean.copy())[:M]
kerngen = lambda: ekernels.RBF(Q, ARD=True)

In [ ]:
X_std_full = np.array([0.1**0.5*np.eye(Q) for _ in range(N)])
mfull = GPflow.gplvm.BayesianGPLVM(X_mean=X_mean.copy(), X_std=X_std_full, Y=Y,
                                kern=kerngen(), M=M, Z=Z.copy())
mfull.likelihood.variance = 0.01

In [ ]:
# Check the new full q(x) has the same value when diagonal is used
np.set_printoptions(suppress=True, precision=3)
mfull_ll = list()
mdiag_ll = list()
def evalDiagBound(free_state):
    ''' Callback for diag model optimizer. Also evaluates bound for full model '''    
    oldstate = mdiag.get_free_state()
    mdiag.set_state(free_state)
    mdiag_ll.append(mdiag.compute_log_likelihood())
    d = mdiag.get_parameter_dict()
    # replace diagonal with full covariance
    X_std_fullmodel = np.array([np.diag(mdiag.X_std.value[i,:]) for i in range(N)])
    d['model.X_std'] = X_std_fullmodel
    mfull.set_parameter_dict(d)
    mfull_ll.append(mfull.compute_log_likelihood())
    mdiag.set_state(oldstate)  # and restore

mdiag = GPflow.gplvm.BayesianGPLVM(X_mean=X_mean.copy(), X_std=0.1**0.5*np.ones((N, Q)), Y=Y,
                                kern=kerngen(), M=M, Z=Z.copy())
mdiag.likelihood.variance = 0.01
mdiag.optimize(disp=True, maxiter=100, callback=evalDiagBound)

assert np.allclose(np.array(mfull_ll), np.array(mdiag_ll)), 'Loglikelihood should be same for same q(x)'

In [ ]:
mfull_ll = list()
mdiag_ll = list()
def evalFullBound(free_state):
    ''' Callback for full model optimizer. Also evaluate bound for diagonal model '''
    oldstate = mfull.get_free_state()
    mfull.set_state(free_state)
    mfull_ll.append(mfull.compute_log_likelihood())
    d = mfull.get_parameter_dict()
    # replace full covariance with diagonal
    X_std_diagmodel = np.array([np.diag(mfull.X_std.value[i,:,:]) for i in range(N)])
    d['model.X_std'] = X_std_diagmodel
    mdiag.set_parameter_dict(d)
    mdiag_ll.append(mdiag.compute_log_likelihood())
    mfull.set_state(oldstate)  # and restore
    
mfull.optimize(disp=True, maxiter=100, callback=evalFullBound)

In [ ]:
f, ax = plt.subplots(1,1, figsize=(7,7))
ax.plot(np.array(mfull_ll), label='full')
ax.plot(np.array(mdiag_ll), label='diag')
ax.legend()
ax.set_ylabel('Log likelihood bound')

In [ ]:
print(mfull.get_free_state().shape, mdiag.get_free_state().shape)
print(mfull.get_parameter_dict().keys(), mdiag.get_parameter_dict().keys())

In [ ]:
print(mfull.X_std.value.shape, mdiag.X_std.value.shape)

In [ ]:
X_std_fullmodel = np.array([np.diag(mdiag.X_std.value[i,:]) for i in range(N)])
print(X_std_fullmodel.shape)

In [ ]:
print(mdiag.kern)

In [ ]:
print(mfull.kern)

### Compute and sensitivity to input
Sensitivity is a measure of the importance of each latent dimension. 

In [ ]:
colours = ['b', 'y']
labels = ['diag', 'full']
inputIndices = list()
for i, m in enumerate([mdiag, mfull]):
    kern = m.kern
    sens = np.sqrt(kern.variance.value)/kern.lengthscales.value
    inputIndices.append(np.argsort(sens)[::-1][:2])
    print(sens)
    #fig, ax = plt.subplots()
    plt.bar(np.arange(len(kern.lengthscales.value)) + 0.1 * i , sens, 0.1, color=colours[i], label=labels[i])
    plt.legend()
    plt.title('Sensitivity to latent inputs')

In [ ]:
inputIndices

### Plotting vs PCA
We see that using the 2 more relevant dimensions, the Bayesian GPLVM is able to seperate the
three classes while PCA cannot.

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
def linearSep(X, lbl):
    ''' function to compute linear separability '''
    return accuracy_score(lbl, svm.SVC(kernel='linear').fit(X, lbl).predict(X))
    

In [ ]:
XPCAplot = GPflow.gplvm.PCA_reduce(data['X'], 2)
f, ax = plt.subplots(1,3, figsize=(14,6))
labels=data['Y'].argmax(axis=1)
colors = cm.rainbow(np.linspace(0, 1, len(np.unique(labels))))

ax[0].set_title('PCA')
ax[1].set_xlabel(inputIndices[0][0]); ax[1].set_ylabel(inputIndices[0][1])
ax[1].set_title('Bayesian GPLVM - diag Sep=%.2f' % linearSep(mdiag.X_mean.value, labels))
ax[2].set_xlabel(inputIndices[1][0]); ax[2].set_ylabel(inputIndices[1][1])
ax[2].set_title('Bayesian GPLVM - full Sep=%.2f' % linearSep(mfull.X_mean.value, labels))
for i, c in zip(np.unique(labels), colors):
    ax[0].scatter(XPCAplot[labels==i,0], XPCAplot[labels==i,1], color=c, label=i)
    ax[1].scatter(mdiag.X_mean.value[labels==i,inputIndices[0][0]],
                  mdiag.X_mean.value[labels==i,inputIndices[0][1]], color=c, label=i)
    ax[2].scatter(mfull.X_mean.value[labels==i,inputIndices[1][0]], 
                  mfull.X_mean.value[labels==i,inputIndices[1][1]], color=c, label=i)